In [1]:
%load_ext autoreload
%autoreload 2

# Data Processing

In [2]:
import pandas as pd
import json
from datasets import load_dataset

In [3]:
data = None
with open("../../data/Dialogue-Evaluation-with-BERT/dataset/dev.json") as f:
    raw_rows = f.readlines()

In [4]:
rows = list(map(json.loads, raw_rows))

In [5]:
rows[:100]

[{'id': 0,
  'context': ["Am I too abrupt? Did I scare you? Or you don't want ...",
   "No! I have been expecting this moment for a long time . I mean my answer is' yes'.",
   "Oh, I'm so happy . I will try my best to love you and protect you . I promise."],
  'positive_responses': ['So nice of you.',
   'I will also try my best to love you and protect you.',
   'How can I trust you?',
   "I know that. That's why I want to spend my entire life with you.",
   'I promise that I will always be with you.'],
  'adversarial_negative_responses': ['Default, users can select unlocked cells, and press the TAB key to move between the unlocked cells on a protected worksheet.',
   'Last time my friend has promised me to give me a party.',
   'I am expecting to get good marks in the exam, so I will try my level best.',
   "I am so happy because after a long time I went to my grandmother's house.",
   'I see arising a great movement among women to safeguard themselves.'],
  'random_negative_responses

In [6]:
rows[:100]

[{'id': 0,
  'context': ["Am I too abrupt? Did I scare you? Or you don't want ...",
   "No! I have been expecting this moment for a long time . I mean my answer is' yes'.",
   "Oh, I'm so happy . I will try my best to love you and protect you . I promise."],
  'positive_responses': ['So nice of you.',
   'I will also try my best to love you and protect you.',
   'How can I trust you?',
   "I know that. That's why I want to spend my entire life with you.",
   'I promise that I will always be with you.'],
  'adversarial_negative_responses': ['Default, users can select unlocked cells, and press the TAB key to move between the unlocked cells on a protected worksheet.',
   'Last time my friend has promised me to give me a party.',
   'I am expecting to get good marks in the exam, so I will try my level best.',
   "I am so happy because after a long time I went to my grandmother's house.",
   'I see arising a great movement among women to safeguard themselves.'],
  'random_negative_responses

In [7]:
len(rows[0]["context"])

3

In [9]:
from datasets import load_dataset

In [10]:
dataset = load_dataset("daily_dialog")

In [11]:
for split, split_dataset in dataset.items():
    split_dataset.to_csv(f"daily_dialog-{split}.csv") 

Creating CSV from Arrow format:   0%|          | 0/12 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [12]:
df = pd.read_csv("daily_dialog-train.csv")

In [13]:
def parse_dialog(dialog):
    turns = dialog.replace("\'", "").replace("[", "").replace("]", "").replace('"', "").split("\n")
    return list(map(str.strip, turns))

def parse_label_numbers(label_numbers):
    label_numbers = label_numbers.replace("[", "").replace("]", "").split(" ")
    return list(map(int, label_numbers))

In [14]:
parse_dialog(df.iloc[0]["dialog"])

['Say , Jim , how about going for a few beers after dinner ?',
 'You know that is tempting but is really not good for our fitness .',
 'What do you mean ? It will help us to relax .',
 'Do you really think so ? I dont . It will just make us fat and act silly . Remember last time ?',
 'I guess you are right.But what shall we do ? I dont feel like sitting at home .',
 'I suggest a walk over to the gym where we can play singsong and meet some of our friends .',
 'Thats a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them .',
 'Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too .',
 'Good.Let  s go now .   All right .']

In [15]:
parse_dialog(df.iloc[1234]["dialog"])

['David , would you like something to eat ?   No , Im full .',
 'Do you want something to drink ?   Yes , Id like some coffee .',
 'Sorry , I dont have any coffee .',
 'Thats OK . Ill have a glass of water .',
 'A small glass , or a big one ?   Small please .   Here you are .',
 'Thanks .   Youre welcome .']

In [16]:
parse_dialog(df.iloc[50]["dialog"])

['On , darn . The grocery store is closed .',
 'Yeah , this one closes at ten . We could drive to the 24 - hour store on sixth .',
 'Alright . We are out of everything .',
 'I wish the store close to us was open 24 - house a day .',
 'I know , our schedules are so weird . Sometimes , the little corner store is the only thing within walking distance that ’ s open when we get home .',
 'Yeah , and the prices there are very high .',
 'I know . Three dollars for milk .']

In [17]:
parse_label_numbers(df.iloc[50]["act"])

[1, 3, 4, 1, 1, 1, 1]

In [18]:
parse_dialog(df.iloc[2000]["dialog"])

['I noticed your absence in class this morning .', 'Im sorry , I overslept .']

In [19]:
act_labels = ["dummy", "inform", "question", "directive", "commissive"]
emotion_labels = ["no_emotion", "anger", "disgust", "fear", "happiness", "sadness", "surprise"]

In [20]:
df = pd.read_csv("daily_dialog-train.csv")

In [21]:
df.iloc[0]["act"]

'[3 4 2 2 2 3 4 1 3 4]'

In [39]:
new_rows = []
num_turns = 7

for index, row in df.iterrows():
    parsed_dialog = parse_dialog(row["dialog"])

    if len(parsed_dialog) < num_turns:
        continue
    
    parsed_act = parse_label_numbers(row["act"])
    parsed_emotion = parse_label_numbers(row["emotion"])

    dialog = []
    turns = []
    act = []
    emotion = []

    for i, parsed_turn in enumerate(parsed_dialog[:num_turns]):
        dialog.append(parsed_turn)
        
        act_index = parsed_act[i]
        emotion_index = parsed_emotion[i]
        
        act_label = act_labels[act_index]
        emotion_label = emotion_labels[emotion_index]

        act.append(act_label)
        emotion.append(emotion_label) 
        turns.append(f"{act_label}|{emotion_label} {parsed_turn}")
        
    row["turns"] = turns
    row["act_label"] = act
    row["emotion_label"] = emotion
    row["dialog"] = dialog
        
    new_rows.append(row)

In [40]:
new_df = pd.DataFrame.from_records(new_rows)
# new_df.reset_index()
new_df

,dialog,act,emotion,turns,act_label,emotion_label
0,"[Say , Jim , how about going for a few beers a...",[3 4 2 2 2 3 4 1 3 4],[0 0 0 0 0 0 4 4 4 4],"[directive|no_emotion Say , Jim , how about go...","[directive, commissive, question, question, qu...","[no_emotion, no_emotion, no_emotion, no_emotio..."
1,"[Hey John , nice skates . Are they new ?, Yeah...",[2 1 2 1 1 2 1 3 4],[0 0 0 0 0 6 0 6 0],"[question|no_emotion Hey John , nice skates . ...","[question, inform, question, inform, inform, q...","[no_emotion, no_emotion, no_emotion, no_emotio..."
2,"[Hey Lydia , what are you reading ?, I ’ m loo...",[2 1 2 2 1 1 1 1 2 1],[0 4 0 0 0 4 0 0 0 4],"[question|no_emotion Hey Lydia , what are you ...","[question, inform, question, question, inform,...","[no_emotion, happiness, no_emotion, no_emotion..."
3,[I hear you bought a new house in the northern...,[1 1 2 1 1 1 2 1 1 2 1 1 1],[0 0 0 4 4 4 4 0 4 0 0 4 4],[inform|no_emotion I hear you bought a new hou...,"[inform, inform, question, inform, inform, inf...","[no_emotion, no_emotion, no_emotion, happiness..."
4,"[Hi , Becky , whats up ?, Not much , except th...",[2 1 2 1 2 1 1 1 1 2 1 1],[0 0 0 0 0 0 0 0 0 0 0 4],"[question|no_emotion Hi , Becky , whats up ?, ...","[question, inform, question, inform, question,...","[no_emotion, no_emotion, no_emotion, no_emotio..."
...,...,...,...,...,...,...
5289,[Now lets talk about your interests and hobbie...,[3 1 1 1 2 1 2 1 1 1 1 1],[0 0 0 0 0 0 0 0 0 0 0 0],[directive|no_emotion Now lets talk about your...,"[directive, inform, inform, inform, question, ...","[no_emotion, no_emotion, no_emotion, no_emotio..."
5290,[Hello . 9-1- 1 . Can I help you ? I need th...,[2 1 2 1 3 4 1 3 4],[0 0 0 0 0 0 0 0 0],[question|no_emotion Hello . 9-1- 1 . Can I he...,"[question, inform, question, inform, directive...","[no_emotion, no_emotion, no_emotion, no_emotio..."
5291,[Could you do me a favor ? Sure . What is it...,[3 2 3 2 3 2 1 2 1 2 1 2 1 2 2 1 2 1 4],[0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4],[directive|no_emotion Could you do me a favor ...,"[directive, question, directive, question, dir...","[no_emotion, happiness, no_emotion, no_emotion..."
5292,"[Hello , I bought a pen in your shop just befo...",[1 1 1 2 3 2 1 4 1],[0 4 0 0 0 0 0 0 4],"[inform|no_emotion Hello , I bought a pen in y...","[inform, inform, inform, question, directive, ...","[no_emotion, happiness, no_emotion, no_emotion..."


In [41]:
new_df.head()

,dialog,act,emotion,turns,act_label,emotion_label
0,"[Say , Jim , how about going for a few beers a...",[3 4 2 2 2 3 4 1 3 4],[0 0 0 0 0 0 4 4 4 4],"[directive|no_emotion Say , Jim , how about go...","[directive, commissive, question, question, qu...","[no_emotion, no_emotion, no_emotion, no_emotio..."
1,"[Hey John , nice skates . Are they new ?, Yeah...",[2 1 2 1 1 2 1 3 4],[0 0 0 0 0 6 0 6 0],"[question|no_emotion Hey John , nice skates . ...","[question, inform, question, inform, inform, q...","[no_emotion, no_emotion, no_emotion, no_emotio..."
2,"[Hey Lydia , what are you reading ?, I ’ m loo...",[2 1 2 2 1 1 1 1 2 1],[0 4 0 0 0 4 0 0 0 4],"[question|no_emotion Hey Lydia , what are you ...","[question, inform, question, question, inform,...","[no_emotion, happiness, no_emotion, no_emotion..."
3,[I hear you bought a new house in the northern...,[1 1 2 1 1 1 2 1 1 2 1 1 1],[0 0 0 4 4 4 4 0 4 0 0 4 4],[inform|no_emotion I hear you bought a new hou...,"[inform, inform, question, inform, inform, inf...","[no_emotion, no_emotion, no_emotion, happiness..."
4,"[Hi , Becky , whats up ?, Not much , except th...",[2 1 2 1 2 1 1 1 1 2 1 1],[0 0 0 0 0 0 0 0 0 0 0 4],"[question|no_emotion Hi , Becky , whats up ?, ...","[question, inform, question, inform, question,...","[no_emotion, no_emotion, no_emotion, no_emotio..."


In [42]:
new_df.iloc[0]["dialog"]

['Say , Jim , how about going for a few beers after dinner ?',
 'You know that is tempting but is really not good for our fitness .',
 'What do you mean ? It will help us to relax .',
 'Do you really think so ? I dont . It will just make us fat and act silly . Remember last time ?',
 'I guess you are right.But what shall we do ? I dont feel like sitting at home .',
 'I suggest a walk over to the gym where we can play singsong and meet some of our friends .',
 'Thats a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them .']

 # Prepare for Classification


In [50]:
examples = []

for row_index, row in new_df.iterrows():
    for dialog_index, utterance in enumerate(row["dialog"]):
        example = {}
        act_label = row["act_label"][dialog_index]
        emotion_label = row["emotion_label"][dialog_index]
        example["label"] = f"{act_label}|{emotion_label}"
        example["text"] = utterance
        examples.append(example)
        

In [53]:
classification = pd.DataFrame.from_records(examples)

In [54]:
classification

,label,text
0,directive|no_emotion,"Say , Jim , how about going for a few beers af..."
1,commissive|no_emotion,You know that is tempting but is really not go...
2,question|no_emotion,What do you mean ? It will help us to relax .
3,question|no_emotion,Do you really think so ? I dont . It will just...
4,question|no_emotion,I guess you are right.But what shall we do ? I...
...,...,...
37053,question|surprise,His residence ? You mean the whole palace ?
37054,inform|no_emotion,Yes . The Forbidden City is divided into two p...
37055,inform|surprise,"Oh , unbelievable . This was too luxurious ."
37056,inform|no_emotion,Even the whole country belonged to him . Thats...


In [55]:
labels = classification[["label"]].sort_values("label").drop_duplicates().reset_index(drop=True)

In [56]:
labels_map = dict(list(zip(labels.label, labels.index)))

In [57]:
labels_map

{'commissive|anger': 0,
 'commissive|disgust': 1,
 'commissive|fear': 2,
 'commissive|happiness': 3,
 'commissive|no_emotion': 4,
 'commissive|sadness': 5,
 'commissive|surprise': 6,
 'directive|anger': 7,
 'directive|disgust': 8,
 'directive|fear': 9,
 'directive|happiness': 10,
 'directive|no_emotion': 11,
 'directive|sadness': 12,
 'directive|surprise': 13,
 'inform|anger': 14,
 'inform|disgust': 15,
 'inform|fear': 16,
 'inform|happiness': 17,
 'inform|no_emotion': 18,
 'inform|sadness': 19,
 'inform|surprise': 20,
 'question|anger': 21,
 'question|disgust': 22,
 'question|fear': 23,
 'question|happiness': 24,
 'question|no_emotion': 25,
 'question|sadness': 26,
 'question|surprise': 27}